# Merge files

In [202]:
import glob
import os
import shutil

# Location with subdirectories
src_dir = glob.glob("./22/*/*")

# Location to move images to
#move_here = "./30_test/images"
move_xmls = "./22/labels"

move_dir = "./22/images"

if not os.path.exists(move_dir):
    os.mkdir(move_dir)

if not os.path.exists(move_xmls):
    os.mkdir(move_xmls)
    
for file in src_dir:
    #print(file)
    if file.endswith(".jpg"):
        shutil.move(file, move_dir)
    elif file.endswith(".png"):
        shutil.move(file, move_dir)
    else:
        shutil.move(file, move_xmls)

# CVAT annotaions to pascal xml

In [266]:
import pandas
import numpy
import os
# from tqdm import tqdm_notebook

from xml.etree.ElementTree import parse, Element, SubElement, ElementTree
import xml.etree.ElementTree as ET


#save_root1 = "./traffic_light"
save_root2 = "./21/xmls"


#if not os.path.exists(save_root1):
#    os.mkdir(save_root1)

if not os.path.exists(save_root2):
    os.mkdir(save_root2)


def indent(elem, level=0): #자료 출처 https://goo.gl/J8VoDK
    i = "\n" + level*"    "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "    "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

            
def write_xml(folder, filename, width, height, bbox_list):
    root = Element('annotation')
    SubElement(root, 'folder').text = folder
    SubElement(root, 'filename').text = filename
    SubElement(root, 'path').text = './images' +  filename
    source = SubElement(root, 'source')
    SubElement(source, 'database').text = 'Unknown'

    size = SubElement(root, 'size')
    SubElement(size, 'width').text = str(width)
    SubElement(size, 'height').text = str(height)
    SubElement(size, 'depth').text = '3'

    SubElement(root, 'segmented').text = '0'

    for i in bbox_list:
        obj = SubElement(root, 'object')
        SubElement(obj, 'name').text = i[0]
        SubElement(obj, 'pose').text = 'Unspecified'
        SubElement(obj, 'truncated').text = '0'
        SubElement(obj, 'difficult').text = '0'

        bbox = SubElement(obj, 'bndbox')
        SubElement(bbox, 'xmin').text = str(i[1])
        SubElement(bbox, 'ymin').text = str(i[2])
        SubElement(bbox, 'xmax').text = str(i[3])
        SubElement(bbox, 'ymax').text = str(i[4])

    indent(root)
    tree = ElementTree(root)
    tree.write('./'+folder + '/' + filename.split('.')[0] +'.xml')
    
file_nm = glob.glob('./21/labels/*.xml')
#print(file_nm)   

for root_nm in file_nm:  # file_nm is xml_file_list
    before_xml = ET.parse(root_nm)
    root = before_xml.getroot()
    for child in root:
        if child.tag == 'image':
            file_nm = child.attrib['name']
            width = child.attrib['width']
            height = child.attrib['height']
            sign = 0
            #light = 0
            bbox_list = []
            for i in child:
                image_box = []
                if (i.attrib['label'] == 'traffic_sign'): #add traffic_sign in xml
                    sign += 1
                    image_box.append(i.attrib['label'])
                    image_box.append(i.attrib['xtl'])
                    image_box.append(i.attrib['ytl'])
                    image_box.append(i.attrib['xbr'])
                    image_box.append(i.attrib['ybr'])

                    bbox_list.append(image_box)
                # elif (i.attrib['label'] == 'traffic_light') : #add another class in annotation file
                    #light += 1
                    #image_box.append(i.attrib['label'])
                    #image_box.append(i.attrib['xtl'])
                    #image_box.append(i.attrib['ytl'])
                    #image_box.append(i.attrib['xbr'])
                    #image_box.append(i.attrib['ybr'])
                    #bbox_list.append(image_box)
            if sign > 0:
                write_xml(save_root2, file_nm, width, height, bbox_list)
            #if light > 0:
                #write_xml('./traffic_light', file_nm, width, height, bbox_list)


# Make csv from pascal xml

In [267]:

import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     member[4][0].text,
                     member[4][1].text,
                     member[4][2].text,
                     member[4][3].text
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df
    
labels_df = xml_to_csv('./21/xmls/')
labels_df.to_csv(('./21/labels.csv'), index=None)
print('Successfully converted xml to csv.')



Successfully converted xml to csv.


# Extract images based on csv

In [268]:
# import libraries
import os
import shutil
import csv


# csv path
csv_path = os.path.join('./21', 'labels.csv')

# file root
root_path = os.path.join('./21/images')
#root_path = glob.glob("./images/*/*")
# define the saving path after copying files by making new folder 
save_path = os.path.join('./21')

if not os.path.exists(save_path):
    os.mkdir(save_path)
#os.makedirs(save_path)

df = open(csv_path, 'r')
read = csv.reader(df, delimiter= ',')

flag = 0
for i, row in enumerate(read):
    try:
        if flag == 0:
            flag += 1
            pass
        else:
            type_id = row[3]
            dst = os.path.join(save_path, type_id)
            img = os.path.join(root_path, row[0])
            #print(img)
            if not os.path.exists(dst):
                os.makedirs(dst)
        shutil.copy(os.path.join(root_path, row[0]), dst)
    except:
        pass          

# compare files and delete unmatched

In [288]:
#if image folder is subset of annotaion folder we can delete unique files from xml folder

from pathlib import Path
keep_stems = set(file.stem for file in Path('./20/traffic_sign/').glob('*.jpg'))
delete_paths = [file for file in Path('./20/xmls/').glob('*.xml') if file.stem not in keep_stems]
for file in delete_paths:
    file.unlink()

# csv from cvat annotaion

In [ ]:

import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        #print(root)
        for member in root.findall('image'):
            id_1 = member.get('id')
            name = member.get('name')
            width = member.get('width')
            height = member.get('height')
            #xml_list.append([name, width, height])
        for child in root.findall("image/box"):
            xmin = child.get('xtl')
            ymin = child.get('ytl')
            xmax = child.get('xbr')
            ymax = child.get('ybr')
            label = child.get('label')
                     #int(root.find('size')[0].text),
                     #int(root.find('size')[1].text),
                     #member[0].text,
                     #int(member[4][0].text),
                     #int(member[4][1].text),
                     #int(member[4][2].text),
                     #int(member[4][3].text)
                        
                        
                     
            xml_list.append([name, width, height,label, xmin, ymin, xmax, ymax])
            #ml_list.append([name,label, width, height])
            print(xml_list)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    #column_name = ['filename','label', 'width', 'height']

    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df
labels_df = xml_to_csv('./2/xmls')
labels_df.to_csv(('./2/2_sign.csv'), index=None)
print('Successfully converted xml to csv.')


In [476]:
import pandas as pd
import numpy as np
from lxml import etree
import xml.etree.ElementTree as ET


import csv
save_path = "./test/xmls"
filename = './test/labels.csv'

if not os.path.exists(save_path):
    os.mkdir(save_path)


bbox_list = []


with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:
        count = 0
        #print(row[7])
        filename = row[0]
        width = row[1]
        height = row[2]
        classes = row[3]
        xmin = row[4]
        ymin = row[5]
        xmax = row[6]
        ymax = row[7]  
        
        for i in range(0, 2):
            height = df['height'].iloc[i]
            width = df['width'].iloc[i]
            depth = 3
            annotation = ET.Element('annotation')
            ET.SubElement(annotation, 'folder').text = 'images'
            ET.SubElement(annotation, 'filename').text = filename
            ET.SubElement(annotation, 'path').text = './test/'
            source = ET.SubElement(annotation, 'source')
            ET.SubElement(source, 'database').text = 'Unknown'
            size = ET.SubElement(annotation, 'size')
            ET.SubElement(size, 'width').text = str(width)
            ET.SubElement(size, 'height').text = str(height)
            ET.SubElement(size, 'depth').text = '3'

            #segmented = ET.SubElement(annotation, 'segmented')
            ET.SubElement(annotation, 'segmented').text = '0'
            ob = ET.SubElement(annotation, 'object')
            ET.SubElement(ob, 'name').text = classes
            ET.SubElement(ob, 'pose').text = 'Unspecified'
            ET.SubElement(ob, 'truncated').text = '0'
            ET.SubElement(ob, 'difficult').text = '0'
            bbox = ET.SubElement(ob, 'bndbox')
            ET.SubElement(bbox, 'xmin').text = str(xmin)
            ET.SubElement(bbox, 'ymin').text = str(ymin)
            ET.SubElement(bbox, 'xmax').text = str(xmax)
            ET.SubElement(bbox, 'ymax').text = str(ymax)

            fileName = str(df['filename'].iloc[i])
            tree = ET.ElementTree(annotation)
            tree.write(os.path.join(save_path, filename[:-4] + ".xml"), encoding='utf8')
            count += 1    